In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [3]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [4]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
data = df[['aqi']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled.shape

(19706, 1)

In [8]:
scaled = scaled.reshape((19706,))

In [9]:
scaled.shape

(19706,)

In [10]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [11]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [12]:
# batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [13]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.0131 - root_mean_squared_error: 0.1117 - val_loss: 0.0090 - val_root_mean_squared_error: 0.0951 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0077 - root_mean_squared_error: 0.0880 - val_loss: 0.0079 - val_root_mean_squared_error: 0.0890 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0076 - root_mean_squared_error: 0.0873 - val_loss: 0.0077 - val_root_mean_squared_error: 0.0878 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0077 - root_mean_squared_error: 0.0876 - val_loss: 0.0079 - val_root_mean_squared_error: 0.0891 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0075 - root_mean_squared_error: 0.0869 - val_loss: 0.0077 - val_root_mean_squared_error: 0.0875 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0074 - root_mean_squared_

In [14]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [15]:
poll = np.array(df["aqi"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 5.2749133


In [18]:
forecast

array([[0.24022877, 0.22354439, 0.21311624, ..., 0.21720716, 0.21644929,
        0.21289803],
       [0.30836806, 0.33600563, 0.3195543 , ..., 0.32710153, 0.30966547,
        0.30464032],
       [0.16494909, 0.16173987, 0.15193422, ..., 0.17435831, 0.16584557,
        0.16446003],
       ...,
       [0.37420738, 0.37214786, 0.37886512, ..., 0.35488194, 0.3385924 ,
        0.31674656],
       [0.5059542 , 0.49643424, 0.42948195, ..., 0.45834374, 0.4904409 ,
        0.48449382],
       [0.13087724, 0.14183262, 0.14612871, ..., 0.14299515, 0.15175708,
        0.16496015]], dtype=float32)

In [19]:
testPredict

array([[169.31197, 168.29573, 167.66055, ..., 167.90973, 167.86357,
        167.64726],
       [173.4623 , 175.14569, 174.14365, ..., 174.60333, 173.54132,
        173.23524],
       [164.72672, 164.53125, 163.93399, ..., 165.29982, 164.78133,
        164.69693],
       ...,
       [177.47253, 177.34709, 177.75623, ..., 176.29543, 175.30324,
        173.97263],
       [185.49716, 184.91731, 180.83928, ..., 182.59723, 184.55226,
        184.19003],
       [162.65141, 163.3187 , 163.58038, ..., 163.38951, 163.9232 ,
        164.72739]], dtype=float32)

In [20]:
y_test_true

array([[169.93973, 170.07016, 170.20059, ..., 171.11357, 170.46144,
        170.20059],
       [171.76572, 171.89613, 171.89613, ..., 176.06981, 174.6351 ,
        173.06998],
       [164.07051, 163.02708, 161.98367, ..., 167.07033, 167.33119,
        167.46161],
       ...,
       [175.93938, 176.9828 , 177.89578, ..., 180.63477, 176.46109,
        172.28741],
       [183.11287, 180.76518, 178.02621, ..., 177.11322, 176.06981,
        175.93938],
       [162.6358 , 163.5488 , 164.72264, ..., 160.28812, 161.98367,
        163.67923]], dtype=float32)

In [22]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [23]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 2.4604
RMSE for segment 2: 3.1439
RMSE for segment 3: 3.5621
RMSE for segment 4: 3.8454
RMSE for segment 5: 4.0167
RMSE for segment 6: 4.1584
RMSE for segment 7: 4.2993
RMSE for segment 8: 4.3566
RMSE for segment 9: 4.4311
RMSE for segment 10: 4.4573
RMSE for segment 11: 4.5002
RMSE for segment 12: 4.5339
RMSE for segment 13: 4.5786
RMSE for segment 14: 4.6168
RMSE for segment 15: 4.6384
RMSE for segment 16: 4.6550
RMSE for segment 17: 4.6859
RMSE for segment 18: 4.6793
RMSE for segment 19: 4.6371
RMSE for segment 20: 4.5983
RMSE for segment 21: 4.5854
RMSE for segment 22: 4.6270
RMSE for segment 23: 4.6569
RMSE for segment 24: 4.6539
RMSE for segment 25: 4.7534
RMSE for segment 26: 4.8302
RMSE for segment 27: 4.8695
RMSE for segment 28: 4.8667
RMSE for segment 29: 4.9592
RMSE for segment 30: 4.9863
RMSE for segment 31: 4.9682
RMSE for segment 32: 4.9725
RMSE for segment 33: 5.0260
RMSE for segment 34: 5.0295
RMSE for segment 35: 5.0080
RMSE for segment 36: 4.9959
R

In [21]:
model.save("aqi_1dcnn.keras")